### Recogida y envío de datos del día actual de Consumo y Generación del Ed. SEGAI a ThingsBoard

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time # Para trabajar con fechas y tiempo
import os # Librería que nos permite interactuar con el sistema de archivos
import glob # para buscar archivos en un directorio
import pandas as pd
import requests # Para realizar solicitud HTTP con ThingsBoard 
import json # Formato al que hay que enviar los datos a la platafor IoT ThinsBoard

# Carpeta de donde se guardará el archivo con los datos
download_folder = "C:\\Users\\angel\\Desktop\\MASTER\\TFM"

options = webdriver.ChromeOptions()  # Opciones de navegación con selenium

# Configuraciones de descarga en modo headless
options.add_argument("--headless")  
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")

prefs = {
    "download.default_directory": download_folder,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
    "safebrowsing.disable_download_protection": True,
    "profile.default_content_setting_values.automatic_downloads": 1, # Permitir múltiples descargas
}

options.add_experimental_option("prefs", prefs)

# Para asegurarse de que no se bloqueen las descargas en modo headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Inicializar el driver con las opciones configuradas
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Función para verificar la descarga completa del archivo y guardarlo como el archivo más reciente
def esperar_descarga_completa(extension="xls", timeout=60):
    start_time = time.time()
    while True:
        files = glob.glob(download_folder + f"/*.{extension}")
        if files:
            latest_file = max(files, key=os.path.getctime)  # Obtener el archivo más reciente
            print("Archivo descargado con éxito: " + latest_file)
            print()
            return latest_file
        elif time.time() - start_time > timeout:
            print("Tiempo de espera para la descarga excedido.")
            print()
            return None
        time.sleep(1)


# Función para enviar datos a ThingsBoard
def enviar_a_thingsboard(archivo_reciente):
    # URL del Sevirdor IAAS y Token del Dispositivo en ThinsBoard
    url = "https://greenenergy.iaas.ull.es:8080/api/v1/1s32b4n5k9bribml4yqs/telemetry"
    token = "1s32b4n5k9bribml4yqs"
    
    # Leer el archivo excel
    df = pd.read_excel(archivo_reciente, header=2)
    # Inicializar el contador de datos enviados exitosamente
    envios_exitosos = 0
    # Iterar sobre cada fila del DataFrame y enviar los datos a ThingsBoard
    for index, row in df.iterrows():
        # Construir el objeto de datos con las filas del DataFrame y el tiempo
        data = {
            "ts": pd.to_datetime(row['Time']).timestamp() * 1000, # Formato UNIX
            "values": {
                "PV(W)": row['PV(W)'],
                "Meter(W)": row['Meter(W)'],
                "Load(W)": row['Load(W)']
            }
        }
        headers = {'Content-Type': 'application/json', 'X-Authorization': f"Bearer {token}"}
        response = requests.post(url, data=json.dumps(data), headers=headers)
        # Verificación de datos enviados correctamente
        if response.status_code == 200:
            print(f"Dato enviado correctamente: {data}")
            envios_exitosos += 1
        else:
            print(f"Error al enviar dato: {response.text}")
    
    # Si los datos fueron enviados correctamente, borramos archivo de carpeta de origen
    if envios_exitosos == len(df):
        print()
        print("Todos los datos han sido enviados a ThingsBoard exitosamente. Eliminando archivo...")
        os.remove(archivo_reciente)
        print("Archivo eliminado con éxito.")
    else:
        print("Algunos datos no se pudieron enviar correctamente. El archivo no se eliminará.")
        print()
        
# Inicio de sesión y descarga de datos de Sems Portal
login_url = "https://www.semsportal.com/home/login"
driver.get(login_url)

# Completar el formulario de inicio de sesión
time.sleep(2)  # Esperar a que la página cargue

# Esperar a que la página cargue y localizar la casilla de condiciones de uso y política de privacidad
checkbox = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "readStatement"))
)

# Hacer clic en la casilla de verificación si no está ya seleccionada
if not checkbox.is_selected():
    checkbox.click()

email_input = driver.find_element(By.ID, "username")
password_input = driver.find_element(By.ID, "password")
login_button = driver.find_element(By.ID, "btnLogin")

# Credenciales
your_email = "*********************"
your_password = "********"

email_input.send_keys(your_email)
password_input.send_keys(your_password)
login_button.click()

time.sleep(5)  # Esperar a que la página cargue

# Localizar el elemento por el texto que contiene
plant_link = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'SEGAI - ULL')]"))
)
plant_link.click()

# Esperar a que la página cargue 
time.sleep(5)  # Espera para que todos los elementos se carguen completamente

# Navegar a la URL específica
driver.get("https://www.semsportal.com/powerstation/PowerStatusSnMin/2c6eb65e-dca9-4c6f-810f-769064cc6ca8")

max_retries = 3 # Intentos de descarga de un archivo
archivo_descargado = False  # Indica si el archivo ha sido descargado correctamente

for attempt in range(max_retries):
    try:
        # Espera de overload y spinner de carga no visible
        WebDriverWait(driver, 20).until(EC.invisibility_of_element((By.CSS_SELECTOR, '.el-loading-mask')))
        WebDriverWait(driver, 20).until(EC.invisibility_of_element((By.CSS_SELECTOR, '.el-loading-spinner')))
        
        # Realizamos la acción del clic usando ActionChains
        action = ActionChains(driver)
        export_icon = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".goodwe-station-charts__export.fr"))
        )
        action.move_to_element(export_icon).click().perform()

        archivo_reciente = esperar_descarga_completa(extension="xls")
        if archivo_reciente:
            print(f"Clic en el icono de exportación realizado con éxito y archivo descargado.")
            print()
            enviar_a_thingsboard(archivo_reciente)
            archivo_descargado = True
            break  # Sale del bucle de reintentos si se descarga el archivo con éxito
        else:
            print(f"Intento {attempt + 1} de descarga fallido. Reintentando...")
            print()
    except Exception as e:
        print(f"Ocurrió un error durante el intento {attempt + 1}: {e}")
        print()

    time.sleep(5)  # Espera antes de reintentar

if attempt == max_retries - 1 and not esperar_descarga_completa(extension="xls"):
    print("La descarga del archivo falló después de varios intentos.")

# Cerrar el navegador al finalizar todas las descargas
driver.quit()

Tiempo de espera para la descarga excedido.

Intento 1 de descarga fallido. Reintentando...

Archivo descargado con éxito: C:\Users\angel\Desktop\MASTER\TFM\Plant Power_20240305190342.xls

Clic en el icono de exportación realizado con éxito y archivo descargado.

Dato enviado correctamente: {'ts': 1709596800000.0, 'values': {'PV(W)': 0, 'Meter(W)': -26595, 'Load(W)': 26595}}
Dato enviado correctamente: {'ts': 1709597100000.0, 'values': {'PV(W)': 0, 'Meter(W)': -29273, 'Load(W)': 29273}}
Dato enviado correctamente: {'ts': 1709597400000.0, 'values': {'PV(W)': 0, 'Meter(W)': -30345, 'Load(W)': 30345}}
Dato enviado correctamente: {'ts': 1709597700000.0, 'values': {'PV(W)': 0, 'Meter(W)': -27155, 'Load(W)': 27155}}
Dato enviado correctamente: {'ts': 1709598000000.0, 'values': {'PV(W)': 0, 'Meter(W)': -27715, 'Load(W)': 27715}}
Dato enviado correctamente: {'ts': 1709598300000.0, 'values': {'PV(W)': 0, 'Meter(W)': -28012, 'Load(W)': 28012}}
Dato enviado correctamente: {'ts': 1709598600000.0, 

Dato enviado correctamente: {'ts': 1709617800000.0, 'values': {'PV(W)': 0, 'Meter(W)': -54410, 'Load(W)': 54410}}
Dato enviado correctamente: {'ts': 1709618100000.0, 'values': {'PV(W)': 0, 'Meter(W)': -46629, 'Load(W)': 46629}}
Dato enviado correctamente: {'ts': 1709618400000.0, 'values': {'PV(W)': 0, 'Meter(W)': -28476, 'Load(W)': 28476}}
Dato enviado correctamente: {'ts': 1709618700000.0, 'values': {'PV(W)': 0, 'Meter(W)': -33141, 'Load(W)': 33141}}
Dato enviado correctamente: {'ts': 1709619000000.0, 'values': {'PV(W)': 0, 'Meter(W)': -32108, 'Load(W)': 32108}}
Dato enviado correctamente: {'ts': 1709619300000.0, 'values': {'PV(W)': 0, 'Meter(W)': -33275, 'Load(W)': 33275}}
Dato enviado correctamente: {'ts': 1709619600000.0, 'values': {'PV(W)': 0, 'Meter(W)': -34171, 'Load(W)': 34171}}
Dato enviado correctamente: {'ts': 1709619900000.0, 'values': {'PV(W)': 0, 'Meter(W)': -35282, 'Load(W)': 35282}}
Dato enviado correctamente: {'ts': 1709620200000.0, 'values': {'PV(W)': 0, 'Meter(W)': -